In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random


# # Set a seed for reproducibility
# np.random.seed(42)

# # Initialize a 24x24 matrix with zeros
# relationship_matrix = np.zeros((24, 24), dtype=int)

# # Fill the upper triangle of the matrix with random 0s and 1s
# for i in range(24):
#     for j in range(i + 1, 24):
#         relationship_matrix[i, j] = np.random.choice([0, 1])

# # Make the matrix symmetric
# relationship_matrix = conflict_matrix + conflict_matrix.T

# print(relationship_matrix)


# List of first names and last names
first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]

# Set a seed for reproducibility
random.seed(42)

# Generate 24 unique random names
random_names = set()
while len(random_names) < 24:
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    random_names.add(f"{first_name} {last_name}")

random_names = list(random_names)  # Convert set to list

# Initialize a 24x24 matrix with zeros
relationship_matrix = np.zeros((24, 24), dtype=int)

# Randomly select 40 unique pairs for conflicts
conflict_pairs = set()
while len(conflict_pairs) < 40:
    i = random.randint(0, 23)
    j = random.randint(0, 23)
    if i != j and (i, j) not in conflict_pairs and (j, i) not in conflict_pairs:
        conflict_pairs.add((i, j))

# Assign conflicts to the matrix
for i, j in conflict_pairs:
    relationship_matrix[i, j] = 1

# Make the matrix symmetric
conflict_matrix = relationship_matrix + relationship_matrix.T
print(relationship_matrix)
# Create a DataFrame with the conflict matrix
#conflict_df = pd.DataFrame(relationship_matrix, index=random_names, columns=random_names)

# Display the DataFrame
# print(conflict_df)

# Generate a random relationship matrix for demonstration
#relationship_matrix = np.random.choice([0, 1], size=(24, 24))
np.fill_diagonal(relationship_matrix, 0)  # No one has a problem with themselves

# Define the seating grid dimensions
ROWS, COLS = 4, 6

# Define the loss function
def loss_function(arrangement, relationship_matrix):
    arrangement = arrangement.view(ROWS, COLS)
    loss = torch.tensor(0.0, dtype=torch.float, requires_grad=True)
    for r in range(ROWS):
        for c in range(COLS):
            person = arrangement[r, c].item()
            if r > 0:  # Check the person above
                if relationship_matrix[person, arrangement[r - 1, c].item()] == 1:
                    loss = loss + 1
            if r < ROWS - 1:  # Check the person below
                if relationship_matrix[person, arrangement[r + 1, c].item()] == 1:
                    loss = loss + 1
            if c > 0:  # Check the person to the left
                if relationship_matrix[person, arrangement[r, c - 1].item()] == 1:
                    loss = loss + 1
            if c < COLS - 1:  # Check the person to the right
                if relationship_matrix[person, arrangement[r, c + 1].item()] == 1:
                    loss = loss + 1
    return loss

# Define the neural network
class SeatingNN(nn.Module):
    def __init__(self):
        super(SeatingNN, self).__init__()
        self.fc = nn.Linear(24, 24)

    def forward(self, x):
        x = self.fc(x)
        return x

# Training the model
def train_model(relationship_matrix, num_epochs=1000, learning_rate=0.01):
    model = SeatingNN()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = loss_function

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        # Create an initial random arrangement
        arrangement = torch.randperm(24, dtype=torch.float).requires_grad_(True)
        
        # Forward pass
        output = model(arrangement)
        
        # Ensure output is within valid index range by applying argsort
        sorted_indices = output.argsort(dim=0)
        
        # Calculate loss
        loss = criterion(sorted_indices, torch.tensor(relationship_matrix, dtype=torch.float))
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Get the final arrangement
    with torch.no_grad():
        final_output = model(torch.randperm(24, dtype=torch.float))
        final_arrangement = final_output.argsort(dim=0).detach().numpy()
    return final_arrangement

# Function to map numbers to names and reshape
def map_numbers_to_names(arrangement, names_list, rows, cols):
    mapped_arrangement = np.array([names_list[i] for i in arrangement])
    return mapped_arrangement.reshape(rows, cols)

# Train the model and get the optimal arrangement
optimal_arrangement = train_model(relationship_matrix)

# Map the numbers to names and reshape
mapped_arrangement = map_numbers_to_names(optimal_arrangement, random_names, ROWS, COLS)

print("Optimal arrangement with names:")
print(mapped_arrangement)


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0